In [1]:
from opentripdata import main
import multiprocessing as mp
import asyncio
import aiohttp

In [11]:
async def main_async(dest):
    task = asyncio.create_task(main(dest))
    await task
# if __name__ == "__main__":
attraction_df = await main("Los Angeles")

In [12]:
attraction_df

,name,rate,kinds,desc
0,Los Angeles Police Department Memorial to Fall...,3,"historic,monuments_and_memorials,urban_environ...",The Los Angeles Police Department Memorial to ...
1,Los Angeles City Hall,3,"skyscrapers,architecture,interesting_places","Los Angeles City Hall, completed in 1928, is t..."
2,Crawford Addition,3,"architecture,historic_architecture,interesting...",The Higgins Building is a proto-Modernist conc...
3,Historic Core,3,"historic,historical_places,interesting_places,...",The Historic Core is a district within Downtow...
4,Vibiana,3,"religion,cathedrals,interesting_places","The Cathedral of Saint Vibiana, often called S..."
...,...,...,...,...
95,Jeanne d'Arc,3,"historic,cultural,urban_environment,monuments_...",Jeanne d'Arc (ou Jeanne au sacre) est une stat...
96,South Park Lofts,7,"other,unclassified_objects,interesting_places,...",No Description
97,Gallery 1927 at the Fine Arts Building,3,"cultural,museums,interesting_places,art_galleries",The landmark Fine Arts Building is located at ...
98,824 Building,7,"factories,industrial_facilities,interesting_pl...",Gerry Building is a high-rise building in the ...


In [4]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
# Instantiate CountVectorizer and TfidfTransformer
count_vectorizer = CountVectorizer(max_features=5000, stop_words="english")
tfidf_transformer = TfidfTransformer()


In [10]:
count_matrix = count_vectorizer.fit_transform(attraction_df['desc']).toarray()
# Transform the count matrix to TF-IDF matrix
# tfidf_matrix = tfidf_transformer.fit_transform(count_matrix)


In [15]:
cosine_similarity(count_matrix)

array([[1.        , 0.        , 0.12760875, ..., 0.18727644, 0.09421683,
        0.19166144],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.12760875, 0.        , 1.        , ..., 0.34478458, 0.42664092,
        0.26368433],
       ...,
       [0.18727644, 0.        , 0.34478458, ..., 1.        , 0.22135944,
        0.22515121],
       [0.09421683, 0.        , 0.42664092, ..., 0.22135944, 1.        ,
        0.27812135],
       [0.19166144, 0.        , 0.26368433, ..., 0.22515121, 0.27812135,
        1.        ]])

In [17]:
# Calculate cosine similarity between TF-IDF matrix rows (attractions)
cosine_sim = cosine_similarity(count_matrix)

In [84]:
# Recommend attractions based on user input
user_keywords = "nature,"  # User-provided keywords
user_tfidf = count_vectorizer.transform([user_keywords])
user_tfidf = tfidf_transformer.transform(user_tfidf)

In [85]:
user_tfidf.toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [86]:
# Calculate cosine similarity between user input and attractions
user_similarity = cosine_similarity(user_tfidf, tfidf_matrix).flatten()

# Get indices of attractions sorted by similarity
recommended_indices = user_similarity.argsort()[::-1]

In [95]:
count_vectorizer.transform(attraction_df['desc'])

<100x1074 sparse matrix of type '<class 'numpy.int64'>'
	with 2420 stored elements in Compressed Sparse Row format>

In [88]:
# # Display recommended attractions
# for index in recommended_indices:
#     print(attraction_df[index])
attraction_df.iloc[recommended_indices,:]

,name,rate,kinds,desc
99,Southern California Gas Company Complex,7,"historic_architecture,architecture,interesting...",The Southern California Gas Company Complex is...
36,California Plaza Amphitheater,3,"skyscrapers,architecture,cultural,theatres_and...",No Description
26,Plaza Fire House,3,"architecture,historic_architecture,interesting...",No Description
27,Angels Flight,7,"bridges,architecture,interesting_places,other_...",Angels Flight is a landmark and historic 2 ft ...
28,Angels Flight,7,"architecture,historic_architecture,interesting...",Angels Flight is a landmark and historic 2 ft ...
...,...,...,...,...
69,Los Angeles Central Library,7,"historic_architecture,architecture,interesting...",No Description
70,Citigroup,3,"skyscrapers,architecture,interesting_places,ba...","FourFortyFour South Flower, now named Citigrou..."
71,MTA Building,3,"skyscrapers,architecture,interesting_places",No Description
72,Broadway Theater District,7,"historic,historical_places,cultural,urban_envi...",No Description


In [45]:
attraction_df.iloc[16]['desc']

'The Japanese American National Museum (全米日系人博物館, Zenbei Nikkeijin Hakubutsukan) is located in Los Angeles, California, and dedicated to preserving the history and culture of Japanese Americans. Founded in 1992, it is located in the Little Tokyo area near downtown. The museum is an affiliate within the Smithsonian Affiliations program.'